In [42]:
from config import load_dotenv

In [43]:
data = r"C:\Users\CHINMAYI SURYAMATH\Desktop\ML PROJECT ECOMMERCE\NN_pdf\Data\table1.pdf"

In [44]:
print(data[0].page_content)


AttributeError: 'str' object has no attribute 'page_content'

In [45]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(data)
docs = loader.load()

print(len(docs))


1


In [46]:
print(docs[0].page_content)


COMMERCIAL LINES POLICY- COMMON POLICY DECLARATIONS 
NAUTILUS INSURANCE COMPANY 
Scottsdale, Arizona 
Policy No. NN893805 Transaction Type: Renewal 
Renewal of Policy# NN89 3427 
Rewrite of Policy# ______ _ 
Inspection Ordered: 
l:RlYes 0No This insurance contract is issued pursuant to the 
Maine Insurance Laws by an insurer neither 
licensed by norunder the jurisdiction of the 
Maine Bureau of Insurance. 
Cross Ref. Policy# ______ _ 
NIC Quote# 
Named Insured and Mailing Address 
(No., Street, Town or City, County, State, Zip Code) 
Estate of John S. Norton, Sr. 
Beth Preston 
Agent and Mailing Address 
Agency No. 1803-00 
(No., Street, Town or City, County, State, Zip Code) 
Breckenridge Insurance Services, LLC 
PO Box 9010 
1 Acadia Commons 
Westbrook 
Policy 
ME 04098 
NO FLAT CANCELLATION 
Period: From 08/08/2019 to o 8 / o 8 / 2 o 2 o at 12:01 A.M. Standard Time at your mailing address shown above. 
Business Description: Property Owner 
Form of Business: Individual 
Tax Statel§_ 

TEXT SPLITTER

In [47]:
parameters = {
    "chunk_size": 250,
    "chunk_overlap": 50,
    "separators":["\n\n", "\n", " ", ""],    # Split priority
   "keep_separator":"end",                  # Keep separators at end
    "add_start_index":True

}

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(**parameters)
chunks = text_splitter.split_documents(docs)
print(chunks[0].page_content)



COMMERCIAL LINES POLICY- COMMON POLICY DECLARATIONS 
NAUTILUS INSURANCE COMPANY 
Scottsdale, Arizona 
Policy No. NN893805 Transaction Type: Renewal 
Renewal of Policy# NN89 3427 
Rewrite of Policy# ______ _ 
Inspection Ordered:


EMBEDDING

In [49]:
from langchain_openai import OpenAIEmbeddings
from config import OPENAI_API_KEY

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY,model="text-embedding-3-small")

VECTORSTORE

In [50]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

RETRIVER

In [51]:
retriever1 = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [52]:
retriever2 = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5})

In [58]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

def pipeline(prompt):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_API_KEY)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2 ,
        chain_type="stuff"  # simplest way: stuff retrieved docs into prompt
    )

    query01 = prompt
    #print("Q:", query01)
    print("A:", qa_chain.run(query01),"\n\n")

    



In [61]:
docs = retriever2.get_relevant_documents("Policy number")
for i, d in enumerate(docs):
    print(f"Doc {i}:\n", d.page_content[:500], "\n---\n")


C:\Users\CHINMAYI SURYAMATH\AppData\Local\Temp\ipykernel_10840\114819149.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever2.get_relevant_documents("Policy number")


Doc 0:
 Maine Bureau of Insurance. 
Cross Ref. Policy# ______ _ 
NIC Quote# 
Named Insured and Mailing Address 
(No., Street, Town or City, County, State, Zip Code) 
Estate of John S. Norton, Sr. 
Beth Preston 
Agent and Mailing Address 
Agency No. 1803-00 
---

Doc 1:
 FORM(S) AND FORMS AND ENDORSEMENTS, IF ANY, ISSUED TO FORM A PART THEREOF, COMPLETE THE ABOVE NUMBERED POLICY. 
Includes copyrighted material of Insurance Services Office, Inc., with its permission. 
EXHIBIT 
E001 (02/14) 
---

Doc 2:
 Policy 
ME 04098 
NO FLAT CANCELLATION 
Period: From 08/08/2019 to o 8 / o 8 / 2 o 2 o at 12:01 A.M. Standard Time at your mailing address shown above. 
Business Description: Property Owner 
Form of Business: Individual 
Tax Statel§_ 
---

Doc 3:
 $ _______ _ 
$ _______ _ 
$ _______ _ 
$ _______ _ 
$ _______ _ 
Tax & Fee Schedule TOTAL ADVANCE PREMIUM $ 1 100. 00 
Policy Fee 
Inspection Fee Surplus Lines Tax 
$ 65. oo Minimum & Deposit 150.00 
39 · 45 TOTAL TAXES & FEES $ _____ 2_5_4_._4_

In [56]:
pipeline("""You are an expert pdf reader and text and table data extractor . 
         Extract  policy number from the attached document.
          Policy No. NN893806 """)

Q: You are an expert pdf reader and text and table data extractor . 
         Extract  policy number from the attached document.
          Policy No. NN893806 
A: The policy number extracted from the document is NN893805. 




In [60]:
pipeline("""
You are an expert PDF reader and structured data extractor.

Task:
- Read the provided PDF text.
- Extract the following fields:
  - policy_number
  - policy_holder_name
  - start_date
  - end_date
  - premium_amount
  - tables (if present)
-if you find other important data values include in extraction process
         

"""

)

A: I don't know. 




In [65]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Custom structured prompt
    template = """
    You are an expert PDF reader and structured data extractor.

    Task:
    - Read the provided PDF text.
    - Extract the following fields:
      - policy_number
      - policy_holder_name
      - start_date
      - end_date
      - premium_amount
      - tables (if present)
    - If you find other important values (like business type, tax, fees, agent name, etc.) include them too.

    Output:
    Return ONLY valid JSON. If a field is not found, use null.

    Example format:
    {{
      "policy_number": "",
      "policy_holder_name": "",
      "start_date": "",
      "end_date": "",
      "premium_amount": "",
      "tables": [],
      "other_fields": {{}}
    }}

    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)

    print("A:", result["result"], "\n\n")
    print("Source Docs:\n")
    for i, doc in enumerate(result["source_documents"]):
        print(f"Doc {i}:\n", doc.page_content[:], "\n---\n")

# Run pipeline
pipeline("Extract structured policy details from the document")


A: ```json
{
  "policy_number": "NN893805",
  "policy_holder_name": null,
  "start_date": null,
  "end_date": null,
  "premium_amount": "1100.00",
  "tables": [],
  "other_fields": {
    "transaction_type": "Renewal",
    "previous_policy_number": "NN893427",
    "policy_fee": "65.00",
    "minimum_deposit": "150.00",
    "surplus_lines_tax": null,
    "total_taxes_and_fees": "254.45",
    "insurer": "Nautilus Insurance Company",
    "location": "Scottsdale, Arizona"
  }
}
``` 


Source Docs:

Doc 0:
 FORM(S) AND FORMS AND ENDORSEMENTS, IF ANY, ISSUED TO FORM A PART THEREOF, COMPLETE THE ABOVE NUMBERED POLICY. 
Includes copyrighted material of Insurance Services Office, Inc., with its permission. 
EXHIBIT 
E001 (02/14) 
---

Doc 1:
 Rewrite of Policy# ______ _ 
Inspection Ordered: 
l:RlYes 0No This insurance contract is issued pursuant to the 
Maine Insurance Laws by an insurer neither 
licensed by norunder the jurisdiction of the 
Maine Bureau of Insurance. 
---

Doc 2:
 parties shall

In [66]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Custom structured prompt
    template = """
    You are an expert PDF reader and structured data extractor.

    Task:
    - Read the provided PDF text.
    - Extract the following fields:
      - policy_number
      - policy_holder_name
      - start_date
      - end_date
      - premium_amount
      - tables (if present)
    - If you find other important values (like business type, tax, fees, agent name, etc.) include them too.

    Output:
    Return in JSON format. If a field is not found, use null.

    Example format:
    {{
      "policy_number": "",
      "policy_holder_name": "",
      "start_date": "",
      "end_date": "",
      "premium_amount": "",
      "tables": [],
      "other_fields": {{}}
    }}

    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)

    print("A:", result["result"], "\n\n")
    

# Run pipeline
pipeline("Extract structured policy details from the document")


A: ```json
{
  "policy_number": "NN893805",
  "policy_holder_name": null,
  "start_date": null,
  "end_date": null,
  "premium_amount": "1100.00",
  "tables": [],
  "other_fields": {
    "transaction_type": "Renewal",
    "previous_policy_number": "NN893427",
    "policy_fee": "65.00",
    "minimum_deposit": "150.00",
    "surplus_lines_tax": null,
    "total_taxes_and_fees": "254.45",
    "insurer": "Nautilus Insurance Company",
    "location": "Scottsdale, Arizona"
  }
}
``` 




In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Generalized extraction prompt
    template = """
    You are an expert PDF reader and information extractor.

    Task:
    - Carefully read the provided PDF content.
    - Extract and summarize all important information, including:
      • Key fields (like policy numbers, dates, names, addresses, fees, amounts, etc.)
      • Contextual details
      • Tabular data (represent tables clearly in text or structured form)
      • Any other significant information relevant to the document.

    Present the extracted information in a clear, structured, and human-readable format.
    
    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)


    print("A:", result["result"], "\n\n")
    print("Source docs used:\n")
    for i, doc in enumerate(result["source_documents"]):
        print(f"Doc {i}:\n", doc.page_content[:300], "\n---\n")


# Run pipeline
pipeline("Extract all important information from the document")


A: Here is the extracted and summarized information from the provided PDF content:

### Document Summary

**Title:** EXHIBIT E001 (02/14)

**Disclaimer:** 
- The information is derived from sources believed to be correct but is not guaranteed.
- Interested parties should rely on their own information and judgment.
- Property is sold on an "AS IS, WHERE IS" basis.

### Key Fields

- **Policy Number:** (Not provided, indicated as "______")
- **Inspection Ordered:** Yes (indicated as "l:RlYes") / No (indicated as "0No")
- **Insurance Contract Issuer:** Not licensed by or under the jurisdiction of the Maine Bureau of Insurance.
  
### Financial Information

- **Total Advance Premium:** $1,100.00
- **Policy Fee:** $65.00
- **Inspection Fee:** (Not specified)
- **Surplus Lines Tax:** (Not specified)
- **Minimum & Deposit:** $150.00
- **Total Taxes & Fees:** $254.45 (indicated as "$ _____ 2_5_4_._4_5")

### Additional Notes

- The document includes forms and endorsements that complete the abo

In [73]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Generalized extraction prompt
    template = """
    You are a professional document analyst specializing in extracting structured insights from PDF files.

Task Overview: Carefully review the provided PDF content and extract all relevant information with precision. 
Your goal is to produce a clear, organized summary that is easy to understand and actionable.

Extraction Requirements:

Key Fields: Identify and extract critical data points such as:

Policy numbers, reference IDs, names, addresses, dates, Policy Period, fees, amounts, and contact details and others.

Contextual Information: Summarize any descriptive or narrative content that provides background, intent, or implications.

Other Notables: Highlight any disclaimers, terms, conditions, signatures, or annotations that may be relevant.
 Tabular Data: If tables are present, convert them into structured text or markdown tables with clear headers and values.

Output Format: Present the extracted information in a clean, structured format using headings, 
bullet points, and tables where appropriate. Ensure the summary is human readable and

logically organized.
  
    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)


    print("A:", result["result"], "\n\n")



# Run pipeline
pipeline("Extract all important information from the document")


A: # Extracted Information Summary

## Key Fields
- **Policy Number:** Not provided (indicated as "Policy# ______ _")
- **Reference ID:** Not provided
- **Names:** Not provided
- **Addresses:** Not provided
- **Dates:** Not provided
- **Policy Period:** Not specified
- **Fees and Amounts:**
  - **Total Advance Premium:** $1,100.00
  - **Policy Fee:** $65.00
  - **Inspection Fee:** Not specified
  - **Surplus Lines Tax:** Not specified
  - **Minimum & Deposit:** $150.00
  - **Total Taxes & Fees:** $254.45 (noted as "TOTAL TAXES & FEES $ _____ 2_5_4_._4_5")

## Contextual Information
- The document is a disclaimer from Tranzon Auction Properties, indicating that the information provided is believed to be correct but is not guaranteed.
- The insurance contract is issued under Maine Insurance Laws by an insurer that is not licensed by or under the jurisdiction of the Maine Bureau of Insurance.
- The property is being sold on an "AS IS, WHERE IS" basis, meaning the buyer accepts the propert

In [75]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Generalized extraction prompt
    template = """
    You are an expert PDF reader and information extractor.

Task:

Carefully read the provided PDF content and follow these reasoning steps:

Step 1: Identify and list all key fields such as policy numbers, dates, names, addresses, fees, and amounts.

Step 2: Summarize any contextual or descriptive information that explains the purpose or scope of the document,

Step 3: Locate and extract any tabular data. Represent it clearly using structured text or markdown tables.

Step 4: Cross-check totals and calculations for consistency. Flag any discrepancies.

Step 5: Highlight any other significant details such as disclaimers, conditions, or annotations.

Finally, present all extracted information in a clear, structured, and human-readable format.
  
    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)


    print("A:", result["result"], "\n\n")
    

# Run pipeline
pipeline("Extract all important information from the document")


A: Here is the extracted information from the provided PDF content, organized according to the specified steps:

### Step 1: Key Fields
- **Policy Number**: Not provided (indicated as "Policy# ______ _")
- **Inspection Ordered**: Yes (indicated as "l:RlYes 0No")
- **Total Advance Premium**: $1,100.00
- **Policy Fee**: $65.00
- **Inspection Fee**: Not specified (indicated as "$ _______ _")
- **Surplus Lines Tax**: Not specified (indicated as "$ _______ _")
- **Minimum & Deposit**: $150.00
- **Total Taxes & Fees**: $254.45 (indicated as "$ _____ 2_5_4_._4_5")
- **Date**: Not specified (indicated as "EXHIBIT E001 (02/14)")
- **Disclaimer**: "This information was derived from sources believed correct but is not guaranteed."

### Step 2: Contextual Information
The document appears to be an insurance contract or related document issued under Maine Insurance Laws. It includes a disclaimer stating that the information is believed to be correct but is not guaranteed. The property is being sold 

In [76]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Generalized extraction prompt
    template = """
    Carefully examine the table in the insurance policy document.

Start by identifying the section labeled "PREMIUM" and extract the coverage type and its associated cost.

Next, locate the "TOTAL ADVANCE PREMIUM" and confirm its value.

Then, move to the "Tax & Fee Schedule" section.

For each line item (e.g., Policy Fee, Inspection Fee, Surplus Lines Tax), extract the label and corresponding amount.

Calculate the TOTAL TAXES & FEES by summing these values, and verify it matches the stated total.

Finally, extract the overall TOTAL amount due and ensure it reflects the sum of the advance premium and all fees.
  
    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)


    print("A:", result["result"], "\n\n")
    

# Run pipeline
pipeline("Extract all important information from the document")


A: Here is the extracted information from the insurance policy document:

### PREMIUM Section:
- **Coverage Type**: Not specified in the provided text.
- **Associated Cost**: Not specified in the provided text.

### TOTAL ADVANCE PREMIUM:
- **Value**: $1,100.00

### Tax & Fee Schedule:
1. **Policy Fee**: $65.00
2. **Inspection Fee**: $150.00
3. **Surplus Lines Tax**: $39.45

### TOTAL TAXES & FEES:
- **Calculated Total**: 
  - Policy Fee: $65.00
  - Inspection Fee: $150.00
  - Surplus Lines Tax: $39.45
  - **Total**: $65.00 + $150.00 + $39.45 = $254.45
- **Stated Total**: $254.45 (matches calculated total)

### Overall TOTAL Amount Due:
- **Total Amount Due**: 
  - Advance Premium: $1,100.00
  - Total Taxes & Fees: $254.45
  - **Overall Total**: $1,100.00 + $254.45 = $1,354.45

### Summary:
- **TOTAL ADVANCE PREMIUM**: $1,100.00
- **TOTAL TAXES & FEES**: $254.45
- **Overall TOTAL Amount Due**: $1,354.45

(Note: The coverage type and its associated cost were not provided in the text.) 


In [77]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Generalized extraction prompt
    template =  """
You are an expert document analyst. Extract key financial details from the insurance policy document.

Please provide the following information in a structured format:

1. **PREMIUM Section**
   - Coverage Type
   - Associated Cost

2. **TOTAL ADVANCE PREMIUM**
   - Value

3. **Tax & Fee Schedule**
   - Each line item (label and amount)
   - Calculated Total Taxes & Fees
   - Verify against stated total

4. **Overall TOTAL Amount Due**
   - Confirm it equals Advance Premium + Taxes & Fees


    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)


    print("A:", result["result"], "\n\n")
    

# Run pipeline
pipeline("Extract all important information from the document")


A: Here is the structured extraction of key financial details from the insurance policy document:

1. **PREMIUM Section**
   - **Coverage Type**: Not specified in the provided text.
   - **Associated Cost**: Not specified in the provided text.

2. **TOTAL ADVANCE PREMIUM**
   - **Value**: $1,100.00

3. **Tax & Fee Schedule**
   - **Line Items**:
     - Policy Fee: $65.00
     - Inspection Fee: $39.45
     - Surplus Lines Tax: $150.00
   - **Calculated Total Taxes & Fees**: $254.45
   - **Verify against stated total**: The stated total for Taxes & Fees is $254.45, which matches the calculated total.

4. **Overall TOTAL Amount Due**
   - **Total Amount Due**: Not specified in the provided text.
   - **Confirmation**: To confirm, the Overall TOTAL Amount Due should equal Advance Premium ($1,100.00) + Taxes & Fees ($254.45). 

   - **Total Calculation**: 
     - $1,100.00 (Advance Premium) + $254.45 (Taxes & Fees) = $1,354.45

Please note that the Overall TOTAL Amount Due is not explicitly